In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
n_step = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_step, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_step))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)

for a in dataset:
    print(a)

(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 6,  7,  8,  9],
       [ 2,  3,  4,  5],
       [ 8,  9, 10, 11]])>, <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 7,  8,  9, 10],
       [ 3,  4,  5,  6],
       [ 9, 10, 11, 12]])>)
(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[10, 11, 12, 13],
       [ 4,  5,  6,  7],
       [ 0,  1,  2,  3]])>, <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[11, 12, 13, 14],
       [ 5,  6,  7,  8],
       [ 1,  2,  3,  4]])>)


In [3]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
print(shakespeare_text[:60])

First Citizen:
Before we proceed any further, hear me speak.


In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [6]:
tokenizer.texts_to_sequences(["abcABCdD"])

[[5, 22, 19, 5, 22, 19, 13, 13]]

In [7]:
tokenizer.sequences_to_texts([[5,5,5,5,5,5,13,13]])

['a a a a a a d d']

In [8]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [9]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [10]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
n_step = 100
window_length = n_step + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [12]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [13]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [14]:
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

In [15]:
dataset = dataset.prefetch(1)

In [16]:
for X_batch, y_batch in dataset.take(1):
    print(X_batch.shape, y_batch.shape)

(32, 100, 39) (32, 100)


In [ ]:
model = keras.models.Sequential([ 
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

model.fit(dataset, epochs=10)

In [18]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [19]:
X_new = preprocess(["how r yo"])
y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(y_pred + 1)[0]

'e u   t e h o u'

In [20]:
def next_char(text, temperature):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [21]:
next_char("How are yo", temperature=1)

'u'

In [22]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [23]:
print(complete_text(".", temperature=1))

.

warwick:
that yourseranre to mighteth of my tong


In [24]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_step, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [25]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_step, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [26]:
model = keras.models.Sequential([ 
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")),
])

In [27]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [28]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 6s 12ms/step - loss: 2.6195
Epoch 2/50
313/313 [==============================] - 4s 12ms/step - loss: 2.2313
Epoch 3/50
313/313 [==============================] - 4s 12ms/step - loss: 2.1026
Epoch 4/50
313/313 [==============================] - 4s 12ms/step - loss: 2.0281
Epoch 5/50
313/313 [==============================] - 4s 12ms/step - loss: 1.9798
Epoch 6/50
313/313 [==============================] - 4s 12ms/step - loss: 1.9454
Epoch 7/50
313/313 [==============================] - 4s 12ms/step - loss: 1.9174
Epoch 8/50
313/313 [==============================] - 4s 12ms/step - loss: 1.8954
Epoch 9/50
313/313 [==============================] - 4s 12ms/step - loss: 1.8786
Epoch 10/50
313/313 [==============================] - 4s 12ms/step - loss: 1.8601
Epoch 11/50
313/313 [==============================] - 4s 11ms/step - loss: 1.8504
Epoch 12/50
313/313 [==============================] - 4s 12ms/step - loss: 1.8393
Epoch 13/50
3

In [29]:
stateless_model = keras.models.Sequential([ 
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")),
])

In [30]:
stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [31]:
print(complete_text("t"))

timps
and cable by the not abolisy on such.
thinkin


In [32]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

In [33]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [34]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [35]:
datasets.keys()

dict_keys(['train', 'test', 'unsupervised'])

In [36]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [37]:
for X_batch, y_batch in datasets["train"].batch(2).skip(1).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0 = Negative

Review: This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1 = Positive



In [38]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [39]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 59), dtype=string, numpy=
 array([[b'Mann', b'photographs', b'the', b'Alberta', b'Rocky',
         b'Mountains', b'in', b'a', b'superb', b'fashion', b'and',
         b'Jimmy', b'Stewart', b'and', b'Walter', b'Brennan', b'give',
         b'enjoyable', b'performances', b'as', b'they', b'always',
         b'seem', b'to', b'do', b'But', b'come', b'on', b'Hollywood',
         b'a', b'Mountie', b'telling', b'the', b'people', b'of',
         b'Dawson', b'City', b'Yukon', b'to', b'elect', b'themselves',
         b'a', b'marshal', b'yes', b'a', b'marshal', b'and', b'to', b'e',
         b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
         b'<pad>', b'<pad>', b'<pad>', b'<pad>'],
        [b'This', b'is', b'the', b'kind', b'of', b'film', b'for', b'a',
         b'snowy', b'Sunday', b'afternoon', b'when', b'the', b'rest',
         b'of', b'the', b'world', b'can', b'go', b'ahead', b'with',
         b'its', b'own', b'business', b'as', b'you', b'descend', b'into',

In [40]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [41]:
vocabulary.most_common()[:5]

[(b'<pad>', 214309),
 (b'the', 61137),
 (b'a', 38564),
 (b'of', 33983),
 (b'and', 33431)]

In [42]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [43]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [44]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [45]:
table.lookup(tf.constant([b"This movie was sooooooooooooooooooooooooooooooooooo faaaaaaaaaaaaaaaaaantastic".split()]))

<tf.Tensor: shape=(1, 5), dtype=int64, numpy=array([[   22,    12,    11, 10605, 10583]], dtype=int64)>

In [46]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(batch_size).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [47]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [48]:
embed_size = 128
model = keras.models.Sequential([ 
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 13s 14ms/step - loss: 0.6473 - accuracy: 0.5836
Epoch 2/5
782/782 [==============================] - 10s 13ms/step - loss: 0.4072 - accuracy: 0.8168
Epoch 3/5
782/782 [==============================] - 10s 13ms/step - loss: 0.2450 - accuracy: 0.9046
Epoch 4/5
782/782 [==============================] - 10s 13ms/step - loss: 0.1374 - accuracy: 0.9531
Epoch 5/5
782/782 [==============================] - 10s 13ms/step - loss: 0.1131 - accuracy: 0.9602


In [49]:
import datetime

K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

log_dir = "./my_logs/run_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
cb_tensorboard = keras.callbacks.TensorBoard(log_dir)

history = model.fit(train_set, epochs=5, callbacks=[cb_tensorboard])

Epoch 1/5
782/782 [==============================] - 22s 19ms/step - loss: 0.5371 - accuracy: 0.7200
Epoch 2/5
782/782 [==============================] - 15s 19ms/step - loss: 0.3452 - accuracy: 0.8571
Epoch 3/5
782/782 [==============================] - 15s 19ms/step - loss: 0.1852 - accuracy: 0.9342
Epoch 4/5
782/782 [==============================] - 15s 19ms/step - loss: 0.1303 - accuracy: 0.9536
Epoch 5/5
782/782 [==============================] - 15s 19ms/step - loss: 0.1144 - accuracy: 0.9584


In [50]:
import os

TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [51]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [52]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\saved_model.pb
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\assets\tokens.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.data-00000-of-00001
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.index


In [53]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 7s 8ms/step - loss: 0.5502 - accuracy: 0.7202
Epoch 2/5
782/782 [==============================] - 6s 8ms/step - loss: 0.5141 - accuracy: 0.7474
Epoch 3/5
782/782 [==============================] - 6s 8ms/step - loss: 0.5089 - accuracy: 0.7506
Epoch 4/5
782/782 [==============================] - 6s 8ms/step - loss: 0.5052 - accuracy: 0.7538
Epoch 5/5
782/782 [==============================] - 6s 8ms/step - loss: 0.5023 - accuracy: 0.7554


In [54]:
model.predict(["This movie is fucking terrible", "I saw this movie yesterday, but i can't forgot highlight. It was fantastic"])

array([[0.33905065],
       [0.744422  ]], dtype=float32)

In [55]:
model.predict([
    ["Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ..."],
    ["This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ..."]
])

array([[0.7938686],
       [0.7998453]], dtype=float32)

In [56]:
vocab_size = 100
embed_size = 10

In [57]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(decoder_embeddings, initial_state=encoder_state, sequence_length=sequence_lengths)
y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths], outputs=[y_proba])

In [58]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [61]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

Epoch 1/2
32/32 [==============================] - 2s 73ms/step - loss: 4.6069
Epoch 2/2
32/32 [==============================] - 2s 76ms/step - loss: 4.6013


In [67]:
model = keras.models.Sequential([ 
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

In [69]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_10 (GRU)                (None, None, 10)          660       
                                                                 
 bidirectional (Bidirectiona  (None, None, 20)         1320      
 l)                                                              
                                                                 
Total params: 1,980
Trainable params: 1,980
Non-trainable params: 0
_________________________________________________________________


In [2]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 ==1: max_dims += 1
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p/10000**(2*i/max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p/10000**(2*i//max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    
    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

In [3]:
embed_size = 512
max_steps = 500
vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

In [5]:
Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(keras.layers.Dense(vocab_size, activation="softmax"))(Z)

In [14]:
Q = np.random.rand(2, 50, 512)
V = np.random.rand(2, 80, 512)
multi_attn = keras.layers.MultiHeadAttention(8, 8)
multi_attn(Q, V).shape

TensorShape([2, 50, 512])